In [ ]:
# @title 1. 安裝相依套件 (Install Dependencies)
!pip install -q -U \
  langchain-google-genai \
  langchain-groq \
  langchain-openai \
  langchain-core \
  langchain-community \
  langchain_huggingface \
  langchain-chroma \
  sentence-transformers \
  streamlit \
  pyngrok \
  pypdf \
  python-dotenv \
  beautifulsoup4

print("✅ 環境安裝完成！")

✅ 環境安裝完成！


In [ ]:
# @title 2. 設定 API Key
import os
from google.colab import userdata

try:
    # 嘗試從 Colab Secrets 讀取
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ 已從 Secrets 讀取 Google Gemini API Key")
except:
    # 如果沒有設定 Secrets，則手動輸入
    os.environ["GOOGLE_API_KEY"] = input("請輸入您的 Google Gemini API Key: ")


try:
    # 嘗試從 Colab Secrets 讀取
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("✅ 已從 Secrets 讀取 HuggingFace API Key")
except:
    # 如果沒有設定 Secrets，則手動輸入
    os.environ["HF_TOKEN"] = input("請輸入您的 HuggingFace API Key: ")


try:
    # 嘗試從 Colab Secrets 讀取
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    print("✅ 已從 Secrets 讀取 GROQ API Key")
except:
    # 如果沒有設定 Secrets，則手動輸入
    os.environ["GROQ_API_KEY"] = input("請輸入您的 GROQ API Key: ")

✅ 已從 Secrets 讀取 Google Gemini API Key
✅ 已從 Secrets 讀取 HuggingFace API Key
✅ 已從 Secrets 讀取 GROQ API Key


In [ ]:
# @title 3-1. 爬取台灣常用建築法規資料庫

import requests
from bs4 import BeautifulSoup
import json
import re
import time

def scrape_full_building_code():
    print("🕸️ 正在連線至全國法規資料庫...")

    # 目標：建築技術規則建築設計施工編 (PCode: D0070115)
    # 這是台灣室內設計最相關的法規
    url = "https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=D0070115"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' # 確保中文不亂碼

        if response.status_code != 200:
            print(f"❌ 連線失敗，狀態碼：{response.status_code}")
            return []

        soup = BeautifulSoup(response.text, "html.parser")

        # 尋找所有條文區塊
        # 全國法規資料庫的結構通常是 table 或 div row
        # 我們抓取 class 為 "law-reg-content" 下的每一行
        articles = []

        # 抓取條號 (col-no) 與 內容 (col-data)
        rows = soup.find_all("div", class_="row")

        print(f"🔍 掃描到 {len(rows)} 個區塊，正在解析...")

        for row in rows:
            col_no = row.find("div", class_="col-no")
            col_data = row.find("div", class_="col-data")

            if col_no and col_data:
                article_id = col_no.get_text(strip=True)
                content = col_data.get_text(strip=True)

                # 過濾掉廢話，只留真正的條文 (例如 "第 1 條")
                if "第" in article_id and "條" in article_id:
                    articles.append({
                        "ArticleNo": article_id,
                        "ArticleContent": content
                    })

        # 儲存成 JSON 檔案
        filename = "building_regulations_full.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(articles, f, ensure_ascii=False, indent=4)

        print(f"✅ 爬取完成！共取得 {len(articles)} 條法規。")
        print(f"📂 檔案已儲存為：{filename}")

        # 顯示前 3 條看看
        print("\n--- 預覽前 3 條 ---")
        for art in articles[:3]:
            print(f"{art['ArticleNo']}: {art['ArticleContent'][:50]}...")

        return filename

    except Exception as e:
        print(f"❌ 發生錯誤: {e}")
        return None

# 執行爬蟲
json_file = scrape_full_building_code()

🕸️ 正在連線至全國法規資料庫...
🔍 掃描到 390 個區塊，正在解析...
✅ 爬取完成！共取得 389 條法規。
📂 檔案已儲存為：building_regulations_full.json

--- 預覽前 3 條 ---
本條文有附件第 1 條: 本編建築技術用語，其他各編得適用，其定義如下：一、一宗土地：本法第十一條所稱一宗土地，指一幢或二幢以...
本條文有附件第 2 條: 基地應與建築線相連接，其連接部份之最小長度應在二公尺以上。基地內私設通路之寬度不得小於左列標準：一、...
第 2-1 條: 私設通路長度自建築線起算未超過三十五公尺部分，得計入法定空地面積。...


In [ ]:
# @title 3-2. 建構形家格局相關學說資料庫

import json

def create_comprehensive_fengshui_dataset():
    print("📦 正在構建《陽宅三要》、《陽宅十書》、《魯班經》核心通用資料庫...")

    # 這份資料集專注於「通用性」與「現代格局」，不談複雜的羅盤方位
    dataset = [
        # ===============================================================
        # 🔥 第一單元：廚房篇 (灶) - 養命之源
        # 核心經典：《陽宅三要》
        # ===============================================================
        {
            "Book": "陽宅三要",
            "Topic": "開門見灶",
            "Content": "開門見灶，錢財多耗。大門為納氣之口，灶為財庫之象。若一進門即見瓦斯爐，主財氣外露，家運不聚，且易生口角。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "水火不容",
            "Content": "灶位不可與水槽緊鄰，亦不可相對。灶屬火，水槽屬水，水火相戰，主女主人多病，夫妻易生口角。中間宜有料理台緩衝。"
        },
        {
            "Book": "魯班經",
            "Topic": "推車灶",
            "Content": "廚房不可設於房子最前方（近大門處），名為『推車灶』，主家口不和，運勢退敗。廚房宜藏於屋後。"
        },
        {
            "Book": "陽宅十書",
            "Topic": "橫樑壓灶",
            "Content": "灶台上方不可有橫樑壓頂。灶主食祿，若受壓，主女主人健康受損，家中財運受阻。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "廚房無靠",
            "Content": "瓦斯爐後方不可是窗戶，名為『背後空虛』，主漏財。灶後宜靠實牆，象徵有靠山。"
        },

        # ===============================================================
        # 🚪 第二單元：大門與玄關篇 (門) - 納氣之口
        # 核心經典：《陽宅十書》、《魯班經》
        # ===============================================================
        {
            "Book": "陽宅十書",
            "Topic": "穿堂煞",
            "Content": "大門直通後門，或大門正對落地窗，中間無遮擋，名為『穿堂煞』。氣流直入直出，主財來財去，無法聚財。"
        },
        {
            "Book": "魯班經",
            "Topic": "兩門相對",
            "Content": "二家不可門相對，必主一家退；自家房門不可相對，名為『罵門』，主家中口舌是非多。宜設門簾或屏風化解。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "開門見廁",
            "Content": "大門一開即見廁所門，穢氣迎人，主貴人不臨，財神不入。且主身體多病痛。"
        },
        {
            "Book": "陽宅十書",
            "Topic": "開門見鏡",
            "Content": "玄關不可正對大鏡子。鏡子會反射氣場，將大門吸納之吉氣反射出去，主漏財、運勢不順。"
        },
        {
            "Book": "魯班經",
            "Topic": "拱門煞",
            "Content": "家內門形不可做成圓弧拱門狀，名為『驛馬』或『牛軛』，主家人勞碌奔波，留不住人。"
        },

        # ===============================================================
        # 🛏️ 第三單元：臥室篇 (主) - 養氣之所
        # 核心經典：《陽宅三要》、《魯班經》
        # ===============================================================
        {
            "Book": "魯班經",
            "Topic": "橫樑壓床",
            "Content": "床頭或床鋪上方不可有橫樑。樑壓頭，主頭痛、失眠；樑壓胸，主氣悶；樑壓腳，主腳疾。此為陽宅大忌。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "床頭無靠",
            "Content": "床頭宜實不宜虛。床頭不可靠窗，不可懸空，不可背對房門。無靠主心神不寧，易犯小人，睡眠品質差。"
        },
        {
            "Book": "陽宅十書",
            "Topic": "鏡子對床",
            "Content": "臥室內鏡子不可正對床鋪。半夜起身易受驚嚇，且鏡子攝魂，主神經衰弱，夫妻不和。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "廁沖床",
            "Content": "廁所門不可正對床位。廁所為孤陰穢氣之地，沖頭主頭病，沖腰主腰病。宜改門向或設隱形門。"
        },
        {
            "Book": "魯班經",
            "Topic": "壁刀切床",
            "Content": "房內柱子尖角不可沖射床位，名為『壁刀煞』，主血光、開刀。宜用衣櫃填平或圓弧包覆。"
        },

        # ===============================================================
        # 🚽 第四單元：衛浴篇 - 穢氣之源
        # 核心經典：《陽宅十書》
        # ===============================================================
        {
            "Book": "陽宅十書",
            "Topic": "中宮造廁",
            "Content": "房子正中央（中宮）不可設廁所。中宮如人之心臟，宜淨宜亮。廁入中宮，名為『穢氣入心』，主心臟疾病，家運衰退。"
        },
        {
            "Book": "陽宅三要",
            "Topic": "廁所居高",
            "Content": "若為透天或樓中樓，廁所不可設於神明廳或大門上方，名為『淋頭水』，主後代不敬，運勢受壓。"
        },
        {
            "Book": "陽宅十書",
            "Topic": "廁所對餐桌",
            "Content": "廁所門不可正對餐桌。進食之地宜潔，穢氣直沖，主腸胃疾病，亦影響食慾。"
        },

        # ===============================================================
        # 🏠 第五單元：形煞與其他 (樑、柱、光)
        # 核心經典：《魯班經》
        # ===============================================================
        {
            "Book": "魯班經",
            "Topic": "廳中橫樑",
            "Content": "客廳沙發上方不可有橫樑，主一家之主壓力大，運勢受阻。若無法避開，應用天花板包覆平整。"
        },
        {
            "Book": "陽宅十書",
            "Topic": "天斬煞",
            "Content": "窗外不可見兩棟大樓中間之細縫，名為『天斬煞』，主血光、意外。宜掛山海鎮或厚窗簾遮擋。"
        },
        {
            "Book": "魯班經",
            "Topic": "尖角沖射",
            "Content": "屋內不可有尖銳牆角對沖沙發或座位，主犯小人、口角。宜擺放盆栽化解。"
        },
        {
            "Book": "現代形家",
            "Topic": "沙發無靠",
            "Content": "客廳主沙發背後宜有實牆，象徵有靠山。不可背對大門，不可背對走道，不可背對窗戶，否則主貴人不顯，犯小人。"
        }
    ]

    # 存檔
    filename = "general_fengshui_full.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)

    print(f"✅ 資料庫重構完成！已收錄 {len(dataset)} 條【高通用性】形家風水法則。")
    print(f"📂 檔案位置：{filename}")
    return filename

# 執行建立
json_file = create_comprehensive_fengshui_dataset()

📦 正在構建《陽宅三要》、《陽宅十書》、《魯班經》核心通用資料庫...
✅ 資料庫重構完成！已收錄 22 條【高通用性】形家風水法則。
📂 檔案位置：general_fengshui_full.json


In [ ]:
import json
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

INPUT_FILE = "general_fengshui_full.json"
OUTPUT_FILE = "fengshui_refined_gold.json"

class RefinedFengShui(BaseModel):
    modern_title: str = Field(description="現代化的簡短標題，例如：開門見灶的化解")
    original_text: str = Field(description="古籍的完整原文內容")
    plain_explanation: str = Field(description="詳細的白話文解釋，說明為何不好")
    actionable_advice: str = Field(description="現代室內設計的具體解決方案（如：屏風、隱藏門、綠植）")
    tags: str = Field(description="關鍵字標籤")

def run_comprehensive_refinery():
    print("🏭 啟動 Groq 煉油廠 (處理通用形家資料)...")

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    if not os.environ.get("GROQ_API_KEY"):
         print("❌ 缺 Groq Key")
         return

    # GROQ_MODEL = "llama-3.1-8b-instant"
    GROQ_MODEL = "llama-3.3-70b-versatile"

    llm = ChatGroq(
        temperature=0.1,
        model_name=GROQ_MODEL,
        groq_api_key=os.environ.get("GROQ_API_KEY")
    )

    parser = JsonOutputParser(pydantic_object=RefinedFengShui)
    refined_data = []

    print(f"📦 準備提煉 {len(raw_data)} 條資料...")

    for i, item in enumerate(raw_data):
        print(f"   ⚙️ 加工中 ({i+1}/{len(raw_data)}): {item['Topic']}...")

        template = """
        你是現代室內設計與風水專家。請將以下資料轉化為結構化知識。

        主題：{topic}
        古籍：{book}
        內容：{content}

        任務：
        1. 【解釋】：用白話文解釋此禁忌的邏輯（心理學或氣場學、形學、理則派）。
        2. 【建議】：提供 2-3 種現代裝修、或是風水上的化解法（不要迷信符咒，要用設計手法）。
        3. 【標籤】：提取關鍵字。

        {format_instructions}
        """
        prompt = ChatPromptTemplate.from_template(template, partial_variables={"format_instructions": parser.get_format_instructions()})
        chain = prompt | llm | parser

        try:
            result = chain.invoke({"topic": item['Topic'], "book": item['Book'], "content": item['Content']})
            result['source_book'] = item['Book']
            refined_data.append(result)
        except Exception as e:
            print(f"   ⚠️ 跳過: {e}")

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(refined_data, f, ensure_ascii=False, indent=4)

    print(f"\n✨ 煉油完成！產出 {len(refined_data)} 條黃金資料。")
    return OUTPUT_FILE

# 執行
gold_file = run_comprehensive_refinery()

🏭 啟動 Groq 煉油廠 (處理通用形家資料)...
📦 準備提煉 22 條資料...
   ⚙️ 加工中 (1/22): 開門見灶...
   ⚙️ 加工中 (2/22): 水火不容...
   ⚙️ 加工中 (3/22): 推車灶...
   ⚙️ 加工中 (4/22): 橫樑壓灶...
   ⚙️ 加工中 (5/22): 廚房無靠...
   ⚙️ 加工中 (6/22): 穿堂煞...
   ⚙️ 加工中 (7/22): 兩門相對...
   ⚙️ 加工中 (8/22): 開門見廁...
   ⚙️ 加工中 (9/22): 開門見鏡...
   ⚙️ 加工中 (10/22): 拱門煞...
   ⚙️ 加工中 (11/22): 橫樑壓床...
   ⚙️ 加工中 (12/22): 床頭無靠...
   ⚙️ 加工中 (13/22): 鏡子對床...
   ⚙️ 加工中 (14/22): 廁沖床...
   ⚙️ 加工中 (15/22): 壁刀切床...
   ⚙️ 加工中 (16/22): 中宮造廁...
   ⚙️ 加工中 (17/22): 廁所居高...
   ⚙️ 加工中 (18/22): 廁所對餐桌...
   ⚙️ 加工中 (19/22): 廳中橫樑...
   ⚙️ 加工中 (20/22): 天斬煞...
   ⚙️ 加工中 (21/22): 尖角沖射...
   ⚙️ 加工中 (22/22): 沙發無靠...

✨ 煉油完成！產出 22 條黃金資料。


In [ ]:
import json
import os

def fix_json_keys():
    file_path = "fengshui_refined_gold.json"

    if not os.path.exists(file_path):
        print("❌ 找不到檔案！")
        return

    print(f"🕵️‍♂️ 正在檢查 {file_path} 的內容結構...")

    with open(file_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print("❌ JSON 格式損毀，8B 可能寫入了無效的格式。建議重新生成。")
            return

    if not data:
        print("❌ 檔案是空的！")
        return

    # 1. 偷看第一筆資料長怎樣
    first_item = data[0]
    print("\n📝 第一筆資料的 Keys :")
    print(list(first_item.keys()))

    # 2. 定義常見的錯誤映射表 (根據經驗，8B 常犯這些錯)
    # 左邊是 8B 可能寫錯的，右邊是我們標準的
    key_mapping = {
        "標題": "modern_title",
        "現代標題": "modern_title",
        "Title": "modern_title",
        "topic": "modern_title",

        "原文": "original_text",
        "古籍原文": "original_text",
        "Original_Text": "original_text",

        "解釋": "plain_explanation",
        "白話解釋": "plain_explanation",
        "Explanation": "plain_explanation",

        "建議": "actionable_advice",
        "化解建議": "actionable_advice",
        "Advice": "actionable_advice",

        "標籤": "tags",
        "關鍵字": "tags",
        "Tags": "tags"
    }

    fixed_count = 0
    fixed_data = []

    for item in data:
        new_item = item.copy()

        # 嘗試修復每一個 Key
        for wrong_key, correct_key in key_mapping.items():
            if wrong_key in new_item and correct_key not in new_item:
                new_item[correct_key] = new_item.pop(wrong_key)
                # print(f"   🔧 修復: {wrong_key} -> {correct_key}")

        # 檢查是否修復成功 (是否有標準 Key)
        if "modern_title" in new_item:
            fixed_data.append(new_item)
            fixed_count += 1

    # 3. 儲存修復後的檔案
    if fixed_count > 0:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(fixed_data, f, ensure_ascii=False, indent=4)
        print(f"\n✅ 成功修復並標準化了 {fixed_count} 筆資料！")
        print("👉 現在你可以重新執行 '重建資料庫' 的腳本了。")
    else:
        print("\n⚠️ 無法自動修復。請檢查上方印出的 Keys，8B 可能生成了完全不同的結構。")

# 執行修復
fix_json_keys()

🕵️‍♂️ 正在檢查 fengshui_refined_gold.json 的內容結構...

📝 第一筆資料的 Keys :
['modern_title', 'original_text', 'plain_explanation', 'actionable_advice', 'tags', 'source_book']

✅ 成功修復並標準化了 22 筆資料！
👉 現在你可以重新執行 '重建資料庫' 的腳本了。


In [ ]:
import shutil
import os
import time
import gc # Garbage Collector
import json
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ==========================================
# 1. 強制重置資料庫 (Robust Reset)
# ==========================================
def force_reset_database():
    persist_dir = "./chroma_db"

    print("🧹 開始清理舊資料庫...")

    # 1. 強制釋放記憶體中的 Chroma 物件
    # 這是解決 'readonly database' 的關鍵
    if 'db' in globals():
        del globals()['db']
    gc.collect()

    # 2. 刪除資料夾
    if os.path.exists(persist_dir):
        try:
            shutil.rmtree(persist_dir)
            print("   ✅ 舊資料夾已刪除")
            time.sleep(1) # 等待檔案系統釋放鎖定
        except Exception as e:
            print(f"   ❌ 刪除失敗 (請嘗試重啟 Runtime): {e}")
            return False

    return True

# ==========================================
# 2. 注入所有黃金資料 (Ingest)
# ==========================================
def ingest_all_gold_data():
    # 先執行重置
    if not force_reset_database():
        return

    print("🚀 開始重建黃金資料庫...")

    # 初始化 Embedding
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    all_documents = []

    # --- 讀取風水資料 ---
    fengshui_file = "fengshui_refined_gold.json"
    if os.path.exists(fengshui_file):
        print(f"   📖 讀取風水資料：{fengshui_file}")
        with open(fengshui_file, 'r', encoding='utf-8') as f:
            fs_data = json.load(f)
        for item in fs_data:
            content = f"主題：{item['modern_title']}\n古籍原文：{item['original_text']}\n白話解釋：{item['plain_explanation']}\n專家建議：{item['actionable_advice']}\n關鍵字：{item['tags']}"
            doc = Document(
                page_content=content,
                metadata={"source": item.get('source_book', '通用風水'), "category": "fengshui", "type": "gold"}
            )
            all_documents.append(doc)
    else:
        print(f"   ⚠️ 找不到 {fengshui_file}，跳過。")

    # --- 讀取法規資料 ---
    science_file = "building_regulations_full.json"
    if os.path.exists(science_file):
        print(f"   📖 讀取法規資料：{science_file}")
        with open(science_file, 'r', encoding='utf-8') as f:
            sci_data = json.load(f)
        for item in sci_data:
            # 這裡需要檢查一下 json 結構的 key
            # 如果你是用上面爬蟲抓的，key 應該是 'ArticleNo' 和 'ArticleContent'
            # 如果是用模擬的，可能不一樣，這裡做個防呆
            art_no = item.get('ArticleNo') or item.get('id')
            art_content = item.get('ArticleContent') or item.get('content')

            if art_no and art_content:
                text = f"{art_no}：{art_content}"
                doc = Document(
                    page_content=text,
                    metadata={"source": "建築技術規則", "category": "science", "article_id": art_no}
                )
                all_documents.append(doc)
    else:
        print(f"   ⚠️ 找不到 {science_file}，跳過。")

    # --- 寫入 ChromaDB ---
    if all_documents:
        print(f"   💾 正在寫入 {len(all_documents)} 筆資料到 ChromaDB...")
        try:
            db = Chroma.from_documents(
                documents=all_documents,
                embedding=embeddings,
                persist_directory="./chroma_db"
            )
            print("🎉 資料庫重建完成！")
        except Exception as e:
            print(f"❌ 寫入失敗: {e}")
            print("👉 建議：請點擊上方選單 'Runtime' -> 'Restart session' 後再試一次。")
    else:
        print("❌ 沒有任何資料被寫入。")

# 執行
ingest_all_gold_data()

🧹 開始清理舊資料庫...
🚀 開始重建黃金資料庫...


   📖 讀取風水資料：fengshui_refined_gold.json
   📖 讀取法規資料：building_regulations_full.json
   💾 正在寫入 411 筆資料到 ChromaDB...
🎉 資料庫重建完成！


In [ ]:
# @title 4. 主要程式

## requirements.txt

In [ ]:
%%writefile requirements.txt
streamlit
langchain-chroma
langchain-huggingface
langchain-community
langchain-core
langchain-google-genai
langchain-groq
langchain-openai
huggingface-hub
sentence-transformers
chromadb
tenacity
pydantic
google-api-core
google-generativeai
google-cloud-core
pysqlite3-binary

Overwriting requirements.txt


## styles.py

In [ ]:
%%writefile styles.py
import streamlit as st

def apply_floating_bubble_style():
    st.markdown("""
    <style>
        /* 全局背景：深色漸層，營造神祕科技感 */
        .stApp {
            background: linear-gradient(135deg, #0f0c29, #302b63, #24243e);
            color: #E0E0E0;
            font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
        }

        /* 隱藏 Streamlit 預設的 Header 和 Footer 以求美觀 */
        header {visibility: hidden;}
        footer {visibility: hidden;}

        /* 定義漂浮動畫 */
        @keyframes float {
            0% { transform: translateY(0px); }
            50% { transform: translateY(-10px); }
            100% { transform: translateY(0px); }
        }

        /* 標題樣式 */
        .main-title {
            font-size: 3em;
            font-weight: 700;
            background: -webkit-linear-gradient(#00c6ff, #0072ff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            text-align: center;
            margin-bottom: 30px;
            text-shadow: 0 0 20px rgba(0, 198, 255, 0.3);
        }

        /* 通用氣泡卡片 (Glassmorphism) */
        .bubble-card {
            background: rgba(255, 255, 255, 0.05); /* 極淡的白色 */
            backdrop-filter: blur(12px);           /* 毛玻璃模糊 */
            -webkit-backdrop-filter: blur(12px);
            border-radius: 20px;
            border: 1px solid rgba(255, 255, 255, 0.1);
            padding: 20px;
            margin-bottom: 20px;
            box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
            color: #ffffff;
            transition: all 0.3s ease;
        }

        /* 滑鼠懸停時的效果 */
        .bubble-card:hover {
            background: rgba(255, 255, 255, 0.1);
            box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.5);
            transform: scale(1.02);
        }

        /* 左側：科學藍色氣泡 */
        .sci-bubble {
            border-left: 5px solid #00B4D8;
            animation: float 6s ease-in-out infinite;
        }
        .sci-title {
            color: #00B4D8;
            font-weight: bold;
            font-size: 1.2em;
            margin-bottom: 10px;
            display: flex;
            align-items: center;
        }

        /* 右側：風水金色氣泡 */
        .fs-bubble {
            border-left: 5px solid #FFD700;
            animation: float 6s ease-in-out infinite;
            animation-delay: 1s; /* 錯開動畫時間 */
        }
        .fs-title {
            color: #FFD700;
            font-weight: bold;
            font-size: 1.2em;
            margin-bottom: 10px;
            display: flex;
            align-items: center;
        }

        /* 底部：整合建議紫色氣泡 */
        .mediator-bubble {
            background: rgba(108, 92, 231, 0.1);
            border: 1px solid rgba(108, 92, 231, 0.3);
            border-radius: 25px;
            padding: 30px;
            margin-top: 20px;
        }
        .mediator-title {
            color: #a29bfe;
            font-size: 1.5em;
            text-align: center;
            margin-bottom: 15px;
        }

        /* 按鈕美化 */
        div.stButton > button {
            background: linear-gradient(90deg, #00c6ff 0%, #0072ff 100%);
            color: white;
            border: none;
            border-radius: 50px;
            padding: 10px 24px;
            font-weight: bold;
            transition: 0.3s;
            box-shadow: 0 0 15px rgba(0, 114, 255, 0.5);
        }
        div.stButton > button:hover {
            transform: translateY(-2px);
            box-shadow: 0 0 25px rgba(0, 114, 255, 0.8);
        }

    </style>
    """, unsafe_allow_html=True)

Overwriting styles.py


## logic.py

In [ ]:
%%writefile logic.py
import os
import base64
import urllib.parse
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from google.api_core.exceptions import ResourceExhausted

# 引入各大模型庫
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

# ===========================
# 1. 知識庫
# ===========================
class KnowledgeBase:
    def __init__(self, persist_dir="./chroma_db"):
        self.persist_dir = persist_dir
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        self.vector_db = None

    def load_db(self):
        if os.path.exists(self.persist_dir) and os.path.isdir(self.persist_dir):
            try:
                self.vector_db = Chroma(persist_directory=self.persist_dir, embedding_function=self.embeddings)
                return True
            except Exception as e:
                print(f"❌ 資料庫載入失敗: {e}")
                return False
        else:
            print(f"⚠️ 警告：找不到路徑 {self.persist_dir}，請先執行資料重建腳本！")
            return False

    def get_context(self):
        if not self.vector_db:
            success = self.load_db()
            if not success: return None
        return self.vector_db.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 4, "fetch_k": 20}
        )

# ===========================
# 2. 定義輸出結構
# ===========================
class MediatorOutput(BaseModel):
    verdict: str = Field(description="協調者的最終折衷方案與建議，純文字")
    design_prompt: str = Field(description="給 AI 繪圖模型的英文 Prompt")

# ===========================
# 3. 多代理人核心邏輯
# ===========================
class LogicCore:
    def __init__(self):
        self.kb = KnowledgeBase()
        self.kb.load_db()

    # --- 🏭 LLM 工廠 ---
    def _create_llm(self, provider, api_key, model_name=None):
        if not api_key:
            raise ValueError(f"請輸入 {provider} 的 API Key")

        if provider == "Groq":
            return ChatGroq(
                groq_api_key=api_key,
                model_name=model_name or "llama-3.3-70b-versatile",
                temperature=0.5
            )
        elif provider == "OpenAI":
            return ChatOpenAI(
                api_key=api_key,
                model=model_name or "gpt-4o",
                temperature=0.5
            )
        elif provider == "Gemini":
            return ChatGoogleGenerativeAI(
                google_api_key=api_key,
                model=model_name or "gemini-2.5-flash",
                temperature=0.5
            )
        else:
            raise ValueError("不支援的模型供應商")

    # --- 📚 共用功能：RAG 檢索 ---
    def get_rag_context(self, query):
        retriever = self.kb.get_context()
        docs = retriever.invoke(query) if retriever else []
        context_text = "\n".join([d.page_content for d in docs])
        return context_text

    # --- 👁️ 視覺代理人 ---
    def analyze_image(self, image_bytes, api_key, mime_type="image/jpeg"):
        if not api_key: return "未提供 Gemini Key，無法分析圖片。"

        vision_model = ChatGoogleGenerativeAI(
            model="gemini-3.0-pro",
            temperature=0.7,
            google_api_key=api_key
        )
        b64_string = base64.b64encode(image_bytes).decode("utf-8")
        image_url = f"data:{mime_type};base64,{b64_string}"

        prompt = "請仔細觀察這張室內設計圖片。描述空間佈局，重點包含：窗戶位置、樑柱位置、門的相對位置。請用客觀的純文字描述。"
        message = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": image_url}
                ]
            }
        ]

        response = vision_model.invoke(message)
        return response.content

    # --- 🏗️ Agent 1: 建築科學家 ---
    def run_architect_agent(self, context, query, provider, api_key, model_name):
        llm = self._create_llm(provider, api_key, model_name)

        template = """你現在是【專業的建築科學研究員】。請根據【參考資料】與【使用者需求】，進行科學性的空間檢討。

        【參考資料】：{context}
        【使用者需求】：{query}

        請執行以下思考步驟：
        1. **設計意圖分析**：理解使用者想做什麼。
        2. **科學性的判斷**：
           - 思考改動對空間造成的物理結果（油煙、濕氣、通風）。
           - 檢查【參考資料】，僅引用「直接相關」條文。
           - ⚠️ 重要：若參考資料無關，請忽略並以科學常識回答。

        3. **輸出要求**：請輸出 HTML 格式，包含：
           - `<p><b>📐 設計意圖：</b>...</p>`
           - `<p><b>🔍 科學檢視與法規：</b>...</p>`
           - `<ul><li><b>引用依據：</b>(直接寫出相關法條，若無RAG資料則寫"依據一般建築常規")</li><li><b>具體建議：</b>...</li></ul>`
        """

        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | llm | StrOutputParser()
        return chain.invoke({"context": context, "query": query})

    # --- 🔮 Agent 2: 風水大師 ---
    def run_fengshui_agent(self, context, query, provider, api_key, model_name):
        llm = self._create_llm(provider, api_key, model_name)

        template = """你現在是【資深風水大師】。請根據【參考資料】與【使用者需求】，進行格局吉凶診斷。

        【參考資料】：{context}
        【使用者需求】：{query}

        請執行以下思考步驟：
        1. **煞氣診斷**：判斷格局涉及哪種具體禁忌。
        2. **古籍比對**：
           - 檢查【參考資料】是否有對應原文。
           - ⚠️ 重要：若參考資料無關，請忽略並以內建風水知識回答。

        3. **輸出要求**：請輸出 HTML 格式，包含：
           - `<p><b>🔮 陽宅格局診斷：</b>...</p>`
           - `<p><b>📜 古籍與民俗觀點：</b>...</p>`
           - `<ul><li><b>經典引據：</b>(直接寫出經典名稱，若無RAG資料則寫依據一般風水見解)...</li><li><b>化解之道：</b>...</li></ul>`
        """

        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | llm | StrOutputParser()
        return chain.invoke({"context": context, "query": query})

    # --- 🤝 Agent 3: 協調設計師 ---
    def run_mediator_agent(self, arch_res, fs_res, query, style, provider, api_key, model_name):
        llm = self._create_llm(provider, api_key, model_name)
        parser = JsonOutputParser(pydantic_object=MediatorOutput)

        template = """你是一個多工處理的專家，現在同時擔任【家庭裝修協調者】與【AI 繪圖詠唱師】。

        【使用者需求】：{query}
        【風格偏好】：{style}

        【建築師意見】：{arch_res}
        【風水師意見】：{fs_res}

        請完成以下兩項任務並輸出 JSON：

        ### 任務 1：協調與折衷 (Verdict)
        1. 綜合上述兩位的觀點，指出衝突點，或是組合雙方提出的共識。
        2. 提供一個「具體折衷方案」，同時滿足科學（通風/採光）與風水（心理/避煞）。
        3. 輸出純文字，語氣溫和專業。

        ### 任務 2：視覺化 Prompt 生成 (Design Prompt)
        根據你的折衷方案與使用者的「{style}」風格，撰寫一個英文 Prompt。
        **關鍵要求：**
        - **針對 FLUX 模型優化**
        - **視角設定**：使用 "High-angle 3/4 perspective view" 及 "Cutaway 3D render"。
        - **構圖目標**：畫面稍微旋轉，不要完全正對牆面。
        - 包含風格關鍵字 (e.g., photorealistic, 8k)。
        - 避免不符合現實的景象。

        {format_instructions}
        """

        prompt = ChatPromptTemplate.from_template(template, partial_variables={"format_instructions": parser.get_format_instructions()})
        chain = prompt | llm | parser
        return chain.invoke({"arch_res": arch_res, "fs_res": fs_res, "query": query, "style": style})

    # --- 🎨 繪圖工具 ---
    def generate_image_via_pollinations(self, prompt):
        """
        使用 Pollinations.ai 免費生成圖片
        """
        import urllib.parse
        import random  # 新增 random 以生成隨機種子

        try:
            # 1. 對 Prompt 進行 URL 編碼，避免特殊字符導致連結失效
            encoded_prompt = urllib.parse.quote(prompt)

            # 2. 生成隨機種子 (Seed)，確保每次生成的構圖都不一樣
            seed = random.randint(0, 99999)

            # 3. 建立 URL
            # model=flux-realism: 指定 2025 更強的真實感模型
            # seed={seed}: 固定隨機性
            # nologo=true: 嘗試隱藏浮水印
            url = f"https://image.pollinations.ai/prompt/{encoded_prompt}?model=flux-realism&width=1024&height=768&seed={seed}&nologo=true"

            return url
        except Exception as e:
            print(f"Pollinations Error: {e}")
            return None

    def generate_image_from_hf(self, prompt, hf_token):
        if not hf_token: return None
        from huggingface_hub import InferenceClient
        client = InferenceClient(model="black-forest-labs/FLUX.1-schnell", token=hf_token)
        try:
            image = client.text_to_image(prompt)
            return image
        except Exception as e:
            print(f"HF Error: {e}")
            return None

Overwriting logic.py


## app.py

In [ ]:
%%writefile app.py
import sys
import os

try:
    __import__('pysqlite3')
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
except ImportError:
    pass

import streamlit as st
import styles
from logic import LogicCore
import time

st.set_page_config(
    page_title="HarmonyChainSpace",
    layout="wide",
    page_icon="☯️"
)
styles.apply_floating_bubble_style()

# 初始化核心
if "core" not in st.session_state:
    st.session_state.core = LogicCore()

core = st.session_state.core

# 初始化 Session State
if "arch_result" not in st.session_state: st.session_state.arch_result = None
if "fs_result" not in st.session_state: st.session_state.fs_result = None
if "mediator_result" not in st.session_state: st.session_state.mediator_result = None
if "last_request" not in st.session_state: st.session_state.last_request = 0
if "prompt_content" not in st.session_state: st.session_state.prompt_content = ""


st.markdown('<div class="main-title">☯️ HarmonyChainSpace · 雙軌系統 </div>', unsafe_allow_html=True)

# ==========================================
# 🎛️ 側邊欄：設定與 API Key
# ==========================================
with st.sidebar:
    st.header("🔑 模型與工具設定")

    # 1. 選擇供應商
    provider = st.selectbox("1. 選擇推理大腦", ["Groq", "OpenAI", "Gemini"])

    # 2. 根據選擇顯示對應的 Key 輸入框與 2025 最新模型
    api_key = ""
    model_name = ""

    if provider == "Groq":
        api_key = st.text_input("Groq API Key", type="password", help="推薦 Llama 3.3")
        model_name = st.selectbox("模型版本", ["llama-3.3-70b-versatile", "llama-3.1-8b-instant"])

    elif provider == "OpenAI":
        api_key = st.text_input("OpenAI API Key", type="password", help="推薦 GPT-4o")
        model_name = st.selectbox("模型版本", ["gpt-4o", "gpt-4o-mini"])

    elif provider == "Gemini":
        api_key = st.text_input("Google AI Studio Key", type="password", help="推薦 Gemini 2.5 Flash")
        model_name = st.selectbox("模型版本", ["gemini-2.5-flash", "gemini-1.5-pro"])

    st.markdown("---")

    # 3. 視覺與繪圖設定
    st.caption("🎨 視覺與繪圖工具")

    gemini_vision_key = st.text_input("Gemini Vision Key (選填)", type="password", help="若需分析圖片，請填入 Google Key。若上方已選 Gemini 則可留空。")
    if provider == "Gemini" and api_key and not gemini_vision_key:
        gemini_vision_key = api_key

    paint_mode = st.radio(
        "繪圖引擎",
        ["Pollinations (免費/無限)", "Hugging Face (需Token)", "關閉繪圖"],
        help="Pollinations 使用 Flux 模型且完全免費"
    )

    hf_token = ""
    if paint_mode == "Hugging Face (需Token)":
        hf_token = st.text_input("Hugging Face Token", type="password")

    st.markdown("---")

    if st.button("🔄 重置系統狀態"):
        for key in ["arch_result", "fs_result", "mediator_result", "prompt_content"]:
            if key in st.session_state: del st.session_state[key]
        st.rerun()

    uploaded_file = st.file_uploader("上傳空間實景", type=["jpg", "png", "jpeg"])
    image_desc = ""
    if uploaded_file:
        st.image(uploaded_file, caption="原始空間", use_container_width=True)
        if st.button("👁️ 分析圖片"):
            if not gemini_vision_key:
                st.error("❌ 請輸入 Gemini Vision Key 才能進行視覺分析")
            else:
                with st.spinner("Gemini 3.0 Pro 正在觀察圖片細節..."):
                    image_bytes = uploaded_file.getvalue()
                    image_desc = core.analyze_image(image_bytes, gemini_vision_key, uploaded_file.type)
                    st.success("視覺分析完成")
                    with st.expander("筆記"): st.write(image_desc)

    with st.expander("💡 靈感圖書館 (點擊自動帶入)"):
        st.caption("點擊下方按鈕，自動填寫經典的「科學 vs 風水」衝突場景：")
        scenarios = {
            "📏 樓梯法規陷阱\n\n(測試第33/36條)": "我想在客廳做一個旋轉樓梯，為了省空間，樓梯寬度只想做 70 公分。另外為了極簡美感，我不打算裝扶手，或者只裝 50 公分高的隱形玻璃扶手。這樣設計合法嗎？",
            "🔥 水火不容極限\n\n(測試陽宅三要)": "廚房空間很小，我打算把瓦斯爐直接緊貼著水槽（距離 0 公分），而且瓦斯爐的正對面剛好就是冰箱。聽說這在風水上叫『水火不容』，真的很嚴重嗎？",
            "🚽 中宮造廁\n\n(測試第46條/陽宅十書)": "這是一間老透天，唯一的廁所剛好在房子的『正中央』，而且完全沒有對外窗。我想把它擴建成豪華浴室，如果不移位，只裝一個小抽風機符合法規嗎？風水上會有什麼健康影響？",
            "🏠 穿堂煞與採光\n\n(測試陽宅十書)": "為了讓客廳採光更好，我把大門進來的玄關牆全部打掉，現在一開大門就能直接看到最後面的陽台落地窗，風可以直接灌進來。長輩說這是『穿堂煞』會漏財，真的有這麼誇張？",
            "🛌 樑壓床與通風\n\n(測試魯班經/第46條)": "主臥室天花板有一根深度 80 公分的超大橫樑。設計師建議為了空間感，不要做天花板包覆，直接把床頭放在樑下。這樣睡覺會不會有壓迫感？風水上怎麼說？",
            "🚪 門對門罵門煞\n\n(測試魯班經)": "我家主臥室的門打開，剛好正對著對面小孩房的門，兩扇門距離只有 80 公分。聽說這叫『罵門』會導致家庭失和？如果不能改門的位置，可以用裝修手法化解嗎？"
        }
        def set_prompt(text): st.session_state.prompt_content = text
        for label, text in scenarios.items():
            if st.button(label, use_container_width=True): set_prompt(text)

# ==========================================
# 📝 主介面：輸入與執行
# ==========================================

user_input = st.text_area("需求描述", height=150, key="prompt_content")

design_style = st.selectbox("模擬圖風格", [
    "Modern Minimalist (現代極簡)",
    "Industrial Loft (工業風)",
    "Japanese Wabi-sabi (日式寂侘)",
    "Neo-Chinese (新中式)",
    "Creamy & Cozy (溫潤奶油風)",
    "Scandinavian (北歐簡約風)",
    "Modern Luxury (現代輕奢風)",
    "Vintage Bauhaus (復古包浩斯)",
    "Biophilic Design (自然共生風)",
    "Cyberpunk / Neo-Future (賽博龐克風)"
])
style_en = design_style.split("(")[0].strip()

submit_btn = st.button("🚀 啟動 HarmonyChainSpace")

if submit_btn:
    if not api_key:
        st.error(f"❌ 請先在左側輸入 {provider} API Key 才能啟動大腦！")
        st.stop()

    if time.time() - st.session_state.last_request < 5:
        st.warning("⏳ 請勿頻繁操作...")
        st.stop()
    st.session_state.last_request = time.time()

    final_query = user_input + (f"\n(圖片描述：{image_desc})" if image_desc else "")

    with st.status("🤖 HarmonyChainSpace 正在協作中...", expanded=True) as status:

        st.write("📚 RAG 系統正在翻閱《建築技術規則》與《魯班經》...")
        context_text = core.get_rag_context(final_query)

        st.write(f"👷‍♂️ [Agent 1] 建築師正在檢討法規 ({model_name})...")
        try:
            st.session_state.arch_result = core.run_architect_agent(context_text, final_query, provider, api_key, model_name)
        except Exception as e:
            status.update(label="❌ 建築師發生錯誤", state="error")
            st.error(f"建築師錯誤: {e}")
            st.stop()

        st.write(f"🔮 [Agent 2] 風水師正在推算吉凶 ({model_name})...")
        try:
            st.session_state.fs_result = core.run_fengshui_agent(context_text, final_query, provider, api_key, model_name)
        except Exception as e:
            status.update(label="❌ 風水師發生錯誤", state="error")
            st.error(f"風水師錯誤: {e}")
            st.stop()

        st.write(f"🤝 [Agent 3] 協調者正在整合方案 ({model_name})...")
        try:
            mediator_json = core.run_mediator_agent(
                st.session_state.arch_result,
                st.session_state.fs_result,
                final_query,
                style_en,
                provider,
                api_key,
                model_name
            )
            st.session_state.mediator_result = mediator_json
        except Exception as e:
            status.update(label="❌ 協調者發生錯誤", state="error")
            st.error(f"協調者錯誤: {e}")
            st.stop()

        status.update(label="✅ 協作完成！", state="complete", expanded=False)

if st.session_state.arch_result and st.session_state.fs_result:
    col1, col2 = st.columns(2)
    with col1:
        st.markdown(f"""<div class="bubble-card sci-bubble"><div class="sci-title">📐 建築科學研究員</div>{st.session_state.arch_result}</div>""", unsafe_allow_html=True)
    with col2:
        st.markdown(f"""<div class="bubble-card fs-bubble"><div class="fs-title">🔮 風水大師</div>{st.session_state.fs_result}</div>""", unsafe_allow_html=True)

if st.session_state.mediator_result:
    verdict = st.session_state.mediator_result.get("verdict", "")
    img_prompt = st.session_state.mediator_result.get("design_prompt", "")

    st.markdown("<br>", unsafe_allow_html=True)
    st.markdown(f"""<div class="bubble-card mediator-bubble"><div class="mediator-title">🤝 協調設計方案</div><p>{verdict.replace(chr(10), '<br>')}</p></div>""", unsafe_allow_html=True)

    if paint_mode != "關閉繪圖" and img_prompt:
        st.markdown("<br>", unsafe_allow_html=True)
        st.subheader("🎨 AI 模擬效果圖")
        with st.status("🎨 正在繪製模擬圖...", expanded=True) as img_status:
            img = None
            if paint_mode == "Pollinations (免費/無限)":
                st.write("使用 Pollinations (Flux) 引擎...")
                img = core.generate_image_via_pollinations(img_prompt)
            elif paint_mode == "Hugging Face (需Token)":
                st.write("使用 Hugging Face (Flux) 引擎...")
                if not hf_token:
                    st.error("請在左側輸入 Hugging Face Token")
                else:
                    img = core.generate_image_from_hf(img_prompt, hf_token)

            if img:
                img_status.update(label="繪圖完成！", state="complete", expanded=False)
                st.image(img, caption=f"設計模擬圖 ({paint_mode})", use_container_width=True)
                with st.expander("查看 Prompt"): st.code(img_prompt)
            else:
                img_status.update(label="繪圖失敗", state="error")
                st.error("繪圖失敗，請檢查網路或 Token")

    elif paint_mode == "關閉繪圖" and img_prompt:
        with st.expander("查看 AI 生成的繪圖指令 (未執行繪圖)"):
            st.info("已略過繪圖步驟。")
            st.code(img_prompt)

Overwriting app.py


## Streamlit

In [ ]:
# @title 5. 啟動 Streamlit 伺服器
from pyngrok import ngrok
import os
import time
from google.colab import userdata


# 終止舊的通道 (避免重複開啟)
ngrok.kill()

# 設定 ngrok token (如果不設定，sessions 會很短)
# 建議去 ngrok 官網註冊免費帳號，取得 Authtoken
!ngrok config add-authtoken "36KtpysbSGt6wTAmLBA2a7LrYYf_wQvMd5xoiABW22hzkhmN"

# 在背景執行 Streamlit
get_ipython().system_raw('streamlit run app.py &')

# 等待幾秒讓服務啟動
time.sleep(5)

# 開啟通道
try:
    # 建立 HTTP 通道指向 8501 port
    public_url = ngrok.connect(8501).public_url
    print(f"🚀 您的應用程式已上線！請點擊以下連結：")
    print(f"👉 {public_url}")
    print("\n(注意：如果看到 ngrok 的警告頁面，點擊 'Visit Site' 即可)")
except Exception as e:
    print(f"啟動失敗，請檢查 ngrok 設定: {e}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 您的應用程式已上線！請點擊以下連結：
👉 https://unfailed-noncompressively-nada.ngrok-free.dev

(注意：如果看到 ngrok 的警告頁面，點擊 'Visit Site' 即可)


## 小工具

In [ ]:
import shutil
import os
from google.colab import files

def package_final_release():
    print("📦 正在打包 HarmonyChainSpace...")

    # 定義要打包的檔案
    files_to_pack = [
        "app.py",
        "logic.py",
        "styles.py",
        "requirements.txt",
        "chroma_db"
    ]

    deploy_dir = "HarmonyChainSpace_Deploy"
    if os.path.exists(deploy_dir):
        shutil.rmtree(deploy_dir)
    os.makedirs(deploy_dir)

    for item in files_to_pack:
        if os.path.exists(item):
            destination = os.path.join(deploy_dir, item)
            if os.path.isdir(item):
                shutil.copytree(item, destination)
            else:
                shutil.copy(item, destination)
            print(f"   ✅ 已加入: {item}")
        else:
            print(f"   ⚠️ 警告: 找不到 {item}")

    output_filename = "HarmonyChainSpace_v1.0"
    shutil.make_archive(output_filename, 'zip', deploy_dir)
    print(f"🎉 打包完成！請下載: {output_filename}.zip")

    files.download(f"{output_filename}.zip")

package_final_release()

📦 正在打包 HarmonyChainSpace...
   ✅ 已加入: app.py
   ✅ 已加入: logic.py
   ✅ 已加入: styles.py
   ✅ 已加入: requirements.txt
   ✅ 已加入: chroma_db
🎉 打包完成！請下載: HarmonyChainSpace_v1.0.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import time
import google.generativeai as genai
from huggingface_hub import InferenceClient
from langchain_groq import ChatGroq
from google.api_core.exceptions import ResourceExhausted, ServiceUnavailable

def api_health_check():
    print("🏥 開始 API 健康檢查診斷...\n")

    # 1. 檢查 Google Gemini
    print("------------------------------------------------")
    print("🤖 檢查 Google Gemini API ...")

    google_key = os.environ.get("GOOGLE_API_KEY")
    if not google_key:
        print("❌ 未檢測到 GOOGLE_API_KEY，請先設定環境變數！")
    else:
        genai.configure(api_key=google_key)
        try:
            model = genai.GenerativeModel('gemini-2.0-flash')
            # 發送一個極短的請求以節省 Token
            response = model.generate_content("Hi", generation_config={"max_output_tokens": 5})
            print("✅ 狀態正常！(Alive)")
            print(f"   回應測試: {response.text.strip()}")
        except ResourceExhausted:
            print("🔴 狀態異常：額度已耗盡 (429 Resource Exhausted)")
            print("   建議：請等待 1~2 分鐘後再試，或檢查 Google AI Studio 配額。")
        except Exception as e:
            print(f"❌ 發生其他錯誤: {e}")

    # 2. 檢查 Groq

    print("------------------------------------------------")
    print("⚡ 檢查 Groq API (llama-3.3-70b)...")

    groq_key = os.environ.get("GROQ_API_KEY")
    if not groq_key:
        print("❌ 未檢測到 GROQ_API_KEY")
    else:
        try:
            chat = ChatGroq(
                temperature=0,
                model_name="llama-3.3-70b-versatile",
                groq_api_key=groq_key
            )
            start = time.time()
            # 這裡我們使用 invoke 來測試 LangChain 連接
            response = chat.invoke("Hi")
            duration = time.time() - start

            print(f"✅ 狀態正常！耗時: {duration:.2f}s")
            print(f"   (LPU 引擎通常應小於 1 秒)")
        except Exception as e:
            print(f"❌ 錯誤: {e}")
            if "401" in str(e):
                print("🔴 原因：API Key 無效")
            elif "429" in str(e):
                print("🔴 原因：速率限制 (Rate Limit)")

    # 2. 檢查 Hugging Face
    print("------------------------------------------------")
    print("🎨 檢查 Hugging Face API (FLUX.1-schnell)...")

    hf_token = os.environ.get("HF_TOKEN")
    if not hf_token:
        print("❌ 未檢測到 HF_TOKEN，請先設定環境變數！")
    else:
        try:
            client = InferenceClient(model="black-forest-labs/FLUX.1-schnell", token=hf_token)
            # 嘗試生成一張極小的圖片 (測試連接用)
            # 這裡不實際顯示圖片，只要沒報錯就是成功
            client.text_to_image("a dot")
            print("✅ 狀態正常！(Alive)")
        except Exception as e:
            print(f"❌ 發生錯誤: {e}")
            if "429" in str(e):
                print("🔴 狀態異常：請求過於頻繁 (Rate Limit)")
            elif "401" in str(e):
                print("🔴 狀態異常：Token 無效或權限不足")

    print("------------------------------------------------")
    print("診斷結束。")

# 執行檢查
api_health_check()

🏥 開始 API 健康檢查診斷...

------------------------------------------------
🤖 檢查 Google Gemini API ...


❌ 發生其他錯誤: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 10.602659189s.
------------------------------------------------
⚡ 檢查 Groq API (llama-3.3-70b)...
✅ 狀態正常！耗時: 0.17s
   (LPU 引擎通常應小於 1 秒)
--------------------------------------------

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

def inspect_database():
    print("🔍 正在進行資料庫 X 光檢查...")

    persist_dir = "./chroma_db" # 確保這跟你 logic.py 裡的路徑一樣

    if not os.path.exists(persist_dir):
        print(f"❌ 嚴重錯誤：找不到資料夾 {persist_dir}！")
        print("   原因可能：Colab 重啟後檔案被清空，請重新執行 '重建資料庫' 的腳本。")
        return

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

    # 取得資料總數
    count = db._collection.count()
    print(f"📊 資料庫總筆數：{count} 筆")

    if count == 0:
        print("❌ 資料庫是空的！請重新執行 Ingest 腳本。")
    else:
        print("✅ 資料庫看起來正常。")

        # 測試檢索：我們直接問它「樓梯扶手」
        print("\n🧪 測試檢索關鍵字：'瓦斯爐'")
        results = db.similarity_search("廚房瓦斯爐", k=5)

        for i, doc in enumerate(results):
            print(f"   [{i+1}] 來源：{doc.metadata.get('source')} | 內容摘要：{doc.page_content[:50]}...")

# 執行檢查
inspect_database()

🔍 正在進行資料庫 X 光檢查...
📊 資料庫總筆數：411 筆
✅ 資料庫看起來正常。

🧪 測試檢索關鍵字：'瓦斯爐'
   [1] 來源：魯班經 | 內容摘要：主題：開門見灶的化解
古籍原文：廚房不可設於房子最前方（近大門處），名為『推車灶』，主家口不和，運勢...
   [2] 來源：建築技術規則 | 內容摘要：第 206 條：地下建築物內不得存放使用桶裝液化石油氣。瓦斯供氣管路應依左列規定：一、燃氣用具應使用...
   [3] 來源：陽宅三要 | 內容摘要：主題：廚房無靠的化解
古籍原文：瓦斯爐後方不可是窗戶，名為『背後空虛』，主漏財。灶後宜靠實牆，象徵有...
   [4] 來源：陽宅三要 | 內容摘要：主題：開門見灶的化解
古籍原文：開門見灶，錢財多耗。大門為納氣之口，灶為財庫之象。若一進門即見瓦斯爐...
   [5] 來源：建築技術規則 | 內容摘要：第 211 條：地下使用單元等使用瓦斯之場所，均應設置左列瓦斯漏氣自動警報設備：一、瓦斯漏氣探測設備...
